##### 0 - Requirementes and configurations

In [7]:
# Set autoreload
%load_ext autoreload
%autoreload 2

# Python modules
import logging
import pandas as pd

# Data modules
from src.data_handler.crypto_price_fetcher import get_crypto_data
from src.data_handler.news_processor import process_news
from src.data_handler.events_processor import create_events


# Set logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
logger = logging.getLogger(__name__)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 1 - Load and process events data

In [8]:
# Get Prices
btc_data = get_crypto_data('BTC', 'USDT', 'hour', day=1, month=1, year=2019)

# Get News
sample = 1000
news = process_news(sample=sample)

# Create Events
events = create_events(btc_data, news)

2025-03-10 17:09:43,746 - INFO - Loading cached data from data/prices/BTC_USDT_hour_2019-01-01_2260days.csv


Processed 1000 news articles


In [16]:
# Select third row from df events to test_event
n_row = 504
test_event = events.iloc[[n_row]]
test_event

,date,text,timestamp,open,high,low,close,volume,next_t_close,close_7,close_30,close_90,target,diff_perc
593,2019-01-24 08:00:00,Title:\n Bitcoin Lightning Network Adoption Ad...,2019-01-24 08:00:00,3545.35,3547.86,3541.4,3547.67,6664.99,3432.12,3541.311429,3545.106333,3562.472778,short,-3.257067


##### 2 - Agent initiation

In [14]:
import os
import logging
from dotenv import load_dotenv
import pandas as pd
from src.models.agent import LLMAgent

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def update_agent_with_dataframe(agent, df: pd.DataFrame):
    """
    A simple implementation to update the agent with data from a DataFrame.
    This example assumes the DataFrame has columns 'date', 'text', 'target', and 'diff_perc'.
    Adjust as needed to properly update news and price data.
    """
    # Here you could parse the DataFrame and update the agent's memory.
    # For example, add news items based on the 'text' column.
    for _, row in df.iterrows():
        # You could create a news item here and update agent memory.
        # This is just a placeholder for demonstration.
        news_text = row.get("text", "")
        if news_text:
            # For demonstration, we simply add the text to both sensory and short-term memories.
            agent.sensory_memory.add_news(row)
            agent.short_term_memory.add_news(row)
            logger.info(f"Updated news memory with: '{news_text}'")
    # You can add similar updates for price data if applicable.

def main(events):
    try:
        # Load environment variables
        load_dotenv()
        logger.info("Environment variables loaded")
        
        # Initialize the agent using the normal __init__
        logger.info("Initializing agent...")
        agent = LLMAgent(
            api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            api_version=os.environ.get("API_VERSION") or "2023-05-15",
            deployment_name=os.environ.get("LLM_MODEL") or "gpt-35-turbo",
            temperature=0.7
        )
        
        # Update agent with the dataframe data using our inline updater
        logger.info("Updating agent with data...")
        update_agent_with_dataframe(agent, events)
        
        # Print current memory state
        print("\n=== CURRENT MEMORY STATE ===")
        print("Sensory Memory:")
        print(agent.sensory_memory.get_formatted())
        print("\nShort-Term Memory:")
        print(agent.short_term_memory.get_formatted())
        print("\nLong-Term Memory:")
        print(agent.long_term_memory.get_formatted())
        
        if not hasattr(agent, 'llm') or agent.llm is None:
            logger.warning("No LLM configured. Skipping recommendation generation.")
            return
        
        # Make a recommendation
        print("\n=== MAKING RECOMMENDATION ===")
        query = "Should I invest in Bitcoin based on the current news and price data?"
        
        try:
            print("Step 1: Analyzing query...")  # REASON
            analysis = agent.react_step(query)
            print("Analysis:", analysis)
            
            print("\nStep 2: Making recommendation...")  # ACT
            decision = agent.make_recommendation(query)
            
            print(f"\nFinal Recommendation: {decision.recommendation}")
            print(f"Confidence: {decision.confidence}")
            print(f"Reasoning: {decision.reasoning}")
            
            # Simulate feedback using the latest row of the events DataFrame
            print("\n=== FEEDBACK PROCESS ===")
            latest_event = events.iloc[-1]
            actual_target = latest_event["target"]  # expected market action (e.g. 'Long', 'Short', or 'Hold')
            diff_perc = float(latest_event["diff_perc"]) if pd.notna(latest_event["diff_perc"]) else 0.0
            
            # Build the outcome message
            outcome = f"Market moved {diff_perc}% with target '{actual_target}'"
            
            # Determine reward based on the agent's recommendation and market movement
            rec = decision.recommendation.strip().lower()
            if rec == "long":
                reward = 0.9 if diff_perc > 0 else 0.1
            elif rec == "short":
                reward = 0.9 if diff_perc < 0 else 0.1
            elif rec == "hold":
                # For hold, reward high if price movement is minimal; otherwise moderate/low
                reward = 0.9 if abs(diff_perc) < 1 else 0.4
            else:
                reward = 0.5  # default if recommendation is unclear
            
            # Process feedback based on the simulation
            learning = agent.process_feedback(decision.decision_id, outcome, reward)
            print("Learning:", learning)
            
            # Print updated memory after feedback
            print("\n=== UPDATED MEMORY AFTER FEEDBACK ===")
            print("Long-Term Memory:")
            print(agent.long_term_memory.get_formatted())
            print("\nProspective Memory:")
            print(agent.prospective_memory.get_formatted())
            print("\nAutobiographical Memory:")
            print(agent.autobiographical_memory.get_formatted())
            
        except Exception as e:
            logger.error(f"Error during recommendation process: {e}")
            
        # Persist the current memory systems for future runs
        agent.save_memory()
        return agent
        
    except Exception as e:
        logger.error(f"Error in main function: {e}")

##### 3 - Run Agent and get recommendation

In [17]:
if __name__ == "__main__":
    agent = main(test_event)

2025-03-10 17:15:33,642 - INFO - Environment variables loaded
2025-03-10 17:15:33,643 - INFO - Initializing agent...
2025-03-10 17:15:33,655 - INFO - Agent memory loaded from file.
2025-03-10 17:15:33,656 - INFO - Updating agent with data...
2025-03-10 17:15:33,658 - INFO - Updated news memory with: 'Title:
 Bitcoin Lightning Network Adoption Advanced by New Bitfury Product Suite

Content:
 The lightning network is still under development but that hasn't stopped the Bitfury Group from launching a new product suite designed to make the system easier for merchants and consumers to use.
Wallets, eCommerce Software, Terminals and Nodes
Lightning network has been hailed as the savior of Bitcoin, bringing its transaction speeds to equal or exceed some of the alternative cryptocurrencies that have emerged over the past couple of years. Rapid and low cost transactions would no doubt to wonders for Bitcoin adoption in everyday life but so far progress has been slow.
Blockchain technology provid


=== CURRENT MEMORY STATE ===
Sensory Memory:
## SENSORY MEMORY (CURRENT MARKET CONDITIONS)

### Latest News:
- 2019-01-24 06:00: Title:
 Bitcoin Value Dropping To Zero? Expert Predicts Fall Of Crypto

Content:
 Jeff Schumacher, the founder of BCG Digital Ventures, has predictedÂ that the value of bitcoin could go as low as zero. As one of the top investors in the crypto arena, Schumacher said that bitcoinÂ couldÂ be at risk especially since there are still doubts over the actual value of the currency.Cryptocurrencies had a wild and challenging 2018. CoinMarketCap estimated that the market lost around $480 billion worth of value. This is after the industry accomplished a record high in 2017. The following yearÂ was a different story.Â Bitcoin, the largest cryptocurrency in the world by market capitalization, took a nosedive.On WednesdayÂ at around 1.15 a.m. London time,Â the cryptocurrency was trading at $3,571. And now Schumacher thinksÂ the value could dive deeper.'I do believe it wi

2025-03-10 17:15:37,125 - INFO - HTTP Request: POST https://canadaeast-aa.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2025-03-10 17:15:37,126 - INFO - Updating memory WorkingMemory: Added thought: '1. Analyze the latest news:
   - The first news article suggests that the value of Bitcoin could go as low as zero, with doubts over the actual value of the currency. This is a negative sentiment towards Bitcoin and raises concerns about its future as a currency. The article also mentions the potential of Bitcoin as a store of value and the importance of the underlying blockchain technology.
   - The second news article highlights the struggle of Bitcoin over the past year, with the withdrawal of a proposal to list a Bitcoin ETF due to the U.S. government shutdown. This is seen as a blow to those who were hoping for increased investment in the market. The article also mentions the lack of direction in the market and t

Analysis: 1. Analyze the latest news:
   - The first news article suggests that the value of Bitcoin could go as low as zero, with doubts over the actual value of the currency. This is a negative sentiment towards Bitcoin and raises concerns about its future as a currency. The article also mentions the potential of Bitcoin as a store of value and the importance of the underlying blockchain technology.
   - The second news article highlights the struggle of Bitcoin over the past year, with the withdrawal of a proposal to list a Bitcoin ETF due to the U.S. government shutdown. This is seen as a blow to those who were hoping for increased investment in the market. The article also mentions the lack of direction in the market and the upcoming Bakkt Bitcoin platform.

2. Analyze the latest prices:
   - There is no specific information about the current price of Bitcoin in the given data.

3. Consider the recent market history:
   - The recent news articles align with the previous news summa

2025-03-10 17:15:40,515 - INFO - HTTP Request: POST https://canadaeast-aa.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2025-03-10 17:15:40,517 - INFO - Updating memory WorkingMemory: Added thought: '1. Analyzing the latest news, we have one article predicting a potential fall in the value of Bitcoin, with the founder of BCG Digital Ventures stating that Bitcoin could go as low as zero. This negative sentiment is concerning.

2. Another article highlights the withdrawal of a proposal to list a Bitcoin exchange-traded fund (ETF) on the Cboe BZX Exchange due to the U.S. government shutdown. This is seen as a blow to those who were hoping for a fresh round of investment into the market.

3. The third article discusses the launch of a new product suite by Bitfury Group aimed at making the Lightning Network easier for merchants and consumers to use. This could potentially increase adoption and usage of Bitcoin.

4. Mov


Final Recommendation: Short
Confidence: 0.8
Reasoning: The negative sentiment, concerns about the future value of Bitcoin, and the withdrawal of the ETF proposal indicate a level of uncertainty in the market. Additionally, the recent positive market movement after the "Hold" recommendation suggests a potential for a short-term decline. However, it is important to continue monitoring the market for any new developments or changes that may impact the decision.

=== FEEDBACK PROCESS ===


2025-03-10 17:15:43,963 - INFO - HTTP Request: POST https://canadaeast-aa.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
2025-03-10 17:15:43,965 - INFO - Updating memory LongTermMemory: Added fact: 'The market may not always behave as expected, and there can be significant deviations from the predicted outcome.'
2025-03-10 17:15:43,966 - INFO - Updating memory ProspectiveMemory: Added consideration: 'Consider the outcome of similar situations to decision 60ff0794-4c93-4253-8b9c-e6532236bc56 (The market may not always behave as expected, and there can be significant deviations from the predicted outcome.)'
2025-03-10 17:15:43,967 - INFO - Agent memory saved successfully.


Learning: 1. Analyze the feedback: The feedback states that the market moved -3.257067314603679% with a target of 'short' after the previous recommendation.

2. Evaluate the significance of the feedback: The negative market movement after the 'Hold' recommendation indicates that the market did not follow the expected pattern. This suggests that the market may be more unpredictable than anticipated.

3. Determine the important fact to remember: 
NEW FACT: The market may not always behave as expected, and there can be significant deviations from the predicted outcome.
CATEGORY: Market Behavior
CONFIDENCE: 0.9

=== UPDATED MEMORY AFTER FEEDBACK ===
Long-Term Memory:
## LONG-TERM MEMORY (MARKET KNOWLEDGE)

### Important Facts:

#### Autobiographical Memory:
- The market moved 0.7870653182198595% with a target of 'Long' after the "Hold" recommendation. (Confidence: 100%)

#### Market Behavior:
- The market may not always behave as expected, and there can be significant deviations from the p

##### 4 - Get memory state

In [18]:
def print_all_agent_memory(agent):
    """
    Print all memory systems of the given agent with clear section headers.
    """
    print("===== AGENT MEMORY SYSTEMS =====\n")
    
    print("=== SENSORY MEMORY ===")
    print(agent.sensory_memory.get_formatted())
    print("\n=== SHORT-TERM MEMORY ===")
    print(agent.short_term_memory.get_formatted())
    print("\n=== PROCEDURAL MEMORY ===")
    print(agent.procedural_memory.get_formatted())
    print("\n=== LONG-TERM MEMORY ===")
    print(agent.long_term_memory.get_formatted())
    print("\n=== AUTOBIOGRAPHICAL MEMORY ===")
    print(agent.autobiographical_memory.get_formatted())
    print("\n=== WORKING MEMORY ===")
    print(agent.working_memory.get_formatted())
    print("\n=== PROSPECTIVE MEMORY ===")
    print(agent.prospective_memory.get_formatted())


# After updating the agent or processing feedback
print_all_agent_memory(agent)

===== AGENT MEMORY SYSTEMS =====

=== SENSORY MEMORY ===
## SENSORY MEMORY (CURRENT MARKET CONDITIONS)

### Latest News:
- 2019-01-24 06:00: Title:
 Bitcoin Value Dropping To Zero? Expert Predicts Fall Of Crypto

Content:
 Jeff Schumacher, the founder of BCG Digital Ventures, has predictedÂ that the value of bitcoin could go as low as zero. As one of the top investors in the crypto arena, Schumacher said that bitcoinÂ couldÂ be at risk especially since there are still doubts over the actual value of the currency.Cryptocurrencies had a wild and challenging 2018. CoinMarketCap estimated that the market lost around $480 billion worth of value. This is after the industry accomplished a record high in 2017. The following yearÂ was a different story.Â Bitcoin, the largest cryptocurrency in the world by market capitalization, took a nosedive.On WednesdayÂ at around 1.15 a.m. London time,Â the cryptocurrency was trading at $3,571. And now Schumacher thinksÂ the value could dive deeper.'I do be